# Navigator

1. Load in mapping table
2. Transform input sheet to data file
3. Compliance analysis
4. Generate output

## Step 1. Load in mapping table

Think of the database as two parts.
The first part is a list of required items derived from the standards.
To make the standards comparable, our unit of analysis is a "disclosure requirement" that can be required or not required in a specific standard (indicated by `req_IFRS` and so on).

The second part of the database contains information about the possible (*hinreichend*) information to comply with the requirements of the standards (indicated by `comp_IFRS` and so on). In principles-based standards, one requirement can have more than one possible information item. When this occurs, we have a list of possible compliance items.

|topic|sub-topic|sub-sub-topic|req_ID|req_IFRS|req_ESRS|req_GRI|comp_IFRS|comp_ESRS|comp_GRI
|---|---|---|---|---|---|---|---|---|---|
|GHG|emissions absolute|Total GHG tCO2eq FY|1|1|1|0|1|1|1|
|GHG|emissions intensity|Scope 1 intensity|2|1|1|1|2,3|2|2,3,4|
|GHG|...|...|...|...|...|...|...|...|...|
|GHG|...|...|n|...|...|...|...|...|...|

This list is linked to the second table. Here, the unit of analysis is an information item linked to a requirement and if that information fulfills the requirement (`req_IFRS = 1`). If it is not required, we write `req_IFRS = n.r.`  

|comp_ID|information_item|req_ID|IFRS|ESRS|GRI|
|---|---|---|---|---|---|
|1|Total GHG emissions in tCO2eq for the fiscal year|1|1|1|n.r.|
|2|Scope 1 emissions per net turnover|2|1|1|1|
|3|Scope 1 emissions per unit of physical or economic output|2|1|0|1|
|4|Scope 1 emissions per any other denominator|2|0|0|1|


We create this mapping based on an Excel file and import it to this sheet.

In [ ]:
import pandas as pd
import numpy as np

mapping = pd.read_excel("221011_mapping_ghg.xlsx")
#mapping.head(5)

We create a dictionary for each standard that contains `req_ID: [comp_IDs, ...]`



In [ ]:
from collections import defaultdict

# for IFRS
mapping_ifrs = defaultdict(list)
for row in mapping[mapping["ifrs_compl"] == 1].itertuples():
  #print(row.req_ID, row.comp_ID)
  mapping_ifrs[row.req_ID].append(row.comp_ID)

# for ESRS
mapping_esrs = defaultdict(list)
for row in mapping[mapping["esrs_compl"] == 1].itertuples():
  mapping_esrs[row.req_ID].append(row.comp_ID)

# for GRI
mapping_gri = defaultdict(list)
for row in mapping[mapping["gri_compl"] == 1].itertuples():
  mapping_gri[row.req_ID].append(row.comp_ID)

In [ ]:
print(mapping_ifrs)

defaultdict(<class 'list'>, {4: [4], 18: [18], 20: [20, 21], 25: [34, 35], 40: [48], 42: [50, 51], 45: [62], 93: [111, 112]})


This results in the following mappings:

In [ ]:
print("IFRS has "+str(len(mapping_ifrs))+" required items.")
print("ESRS has "+str(len(mapping_esrs))+" required items.")
print("GRI has "+str(len(mapping_gri))+" required items.\n\n")

mapping_ifrs

IFRS has 8 required items.
ESRS has 102 required items.
GRI has 14 required items.




defaultdict(list,
            {4: [4],
             18: [18],
             20: [20, 21],
             25: [34, 35],
             40: [48],
             42: [50, 51],
             45: [62],
             93: [111, 112]})

As an example, ESRS E1 AG 44 requires a breakdown of Scope 1 emissions in one of several categories, i.e. leaving more than one possible information.

In [ ]:
mapping_esrs[22]

[26, 27, 28, 29, 30, 31]

Another example, IFRS S2.21 (a) (ii) permits Scope 3 intensity per net turnover or per any unit of physical or economic output. Thus, this requirement is associated with two information items. We can find more information on these by searching for the `comp_ID` in the original mapping table.

In [ ]:
mapping_ifrs[93]

[111, 112]

In [ ]:
for i in mapping_ifrs[93]:
  print(mapping[mapping["comp_ID"] == i].information_item)

112    scope 3 GHG emissions per net turnover
Name: information_item, dtype: object
113    scope 3 GHG emissions per any unit of physical...
Name: information_item, dtype: object


## Step 2. Transform input sheet to data file and assess companies disclosures

We now import the raw data (from the Google doc template) that was collected manually and clean it. Source: https://docs.google.com/spreadsheets/d/1omIPvgw3pKuPWVM1fYy1d3-mDSvjkFLMUAXuKtqpcXY/edit#gid=1302610151

In [ ]:
#import pandas as pd

data_input = pd.read_excel("221019_input_DAX40_AO.xlsx")
#data_input.head(5)

Preparation of the data for further processing and evaluation. We create a data frame in which the `comp_IDs` of the individual companies are displayed in columns. Thus, it can be determined which company fulfils which `comp_IDs`.

In [ ]:
# Data cleaning
data_input.drop(data_input.columns[list(range(5,(len(data_input.columns)),2))], axis=1, inplace=True)
data_input.drop(data_input.columns[[1,2,3]], axis=1, inplace=True)
names = data_input.columns
data_input.rename(columns=dict(zip(data_input.columns, 
         data_input.iloc[0,:].values,)), inplace=True)
data_input.drop([0,1,2], inplace=True)
data_input.reset_index(inplace=True, drop=True)
data_input.rename(columns={data_input.columns[0]: "comp_ID"},inplace=True)
data_input.replace("yes", 1, inplace=True)
data_input.replace("no", 0, inplace=True)

data = pd.DataFrame(data={
    "name" : names[1:] ,
    "isin" : data_input.columns[1:]
})

data.head()

,name,isin
0,Adidas,DE000A1EWWW0
1,Airbus,NL0000235190
2,BASF,DE000BASF111
3,Bayer,DE000BAY0017
4,BMW,DE0005190003


We first collect companies `comp_IDs`, i.e. the list of items they provide (information items).

In [ ]:
list_of_compIDs = []
for isin in data["isin"]:
  compIDs = np.array(data_input.comp_ID * data_input[isin])
  compIDs = compIDs[compIDs != 0]
  list_of_compIDs.append(compIDs)

data["compIDs"] = list_of_compIDs

For each company, then, we create (i) a list of the requirements checked for each standard (`req_ifrs_checked`) and (ii) one compliance value (`req_ifrs`) that represents the percentage of required items fufilled.



In [ ]:
# for IFRS
req_ifrs_checked = []
req_ifrs = []
for isin in data["isin"]:
  temp_list = []
  for key, value in mapping_ifrs.items():
    temp_list.append(len(np.intersect1d(value, list(data[data["isin"] == isin].compIDs))) > 0)

  req_ifrs_checked.append(temp_list)
  req_ifrs.append(sum(temp_list) / len(temp_list))

# for ESRS
req_esrs_checked = []
req_esrs = []
for isin in data["isin"]:
  temp_list = []
  for key, value in mapping_esrs.items():
    temp_list.append(len(np.intersect1d(value, list(data[data["isin"] == isin].compIDs))) > 0)

  req_esrs_checked.append(temp_list)
  req_esrs.append(sum(temp_list) / len(temp_list))

# for GRI
req_gri_checked = []
req_gri = []
for isin in data["isin"]:
  temp_list = []
  for key, value in mapping_gri.items():
    temp_list.append(len(np.intersect1d(value, list(data[data["isin"] == isin].compIDs))) > 0)

  req_gri_checked.append(temp_list)
  req_gri.append(sum(temp_list) / len(temp_list))

data["req_ifrs_checked"] = req_ifrs_checked
data["req_ifrs"] = req_ifrs
data["req_esrs_checked"] = req_esrs_checked
data["req_esrs"] = req_esrs
data["req_gri_checked"]  = req_gri_checked
data["req_gri"] = req_gri

#data.to_excel("221019_data.xlsx")
data.to_json("221019_data.json")
data.head(2)

,name,isin,compIDs,req_ifrs_checked,req_ifrs,req_esrs_checked,req_esrs,req_gri_checked,req_gri
0,Adidas,DE000A1EWWW0,"[1.0, 67.0]","[False, False, False, False, False, False, Fal...",0.00,"[True, False, False, False, False, False, Fals...",0.019608,"[True, False, False, False, False, False, Fals...",0.071429
1,Airbus,NL0000235190,"[1.0, 4.0, 5.0, 9.0, 34.0, 36.0, 40.0, 60.0, 6...","[True, False, False, True, False, False, False...",0.25,"[True, True, True, False, False, False, True, ...",0.166667,"[True, False, False, True, False, False, False...",0.285714


In [ ]:
# check if the format of the lists is ok
tempdatafile = pd.read_json("221019_data.json")
tempdatafile["compIDs"][0]

[1.0, 67.0]

## Step 3. Which requirements are fulfilled how often?

In an additional analysis, we can now evaluate which requirement IDs of the different standards are fulfilled how often.

In [ ]:
#Calculation of the percentages for IFRS relevant req_IDs
perc_comp_ifrs = [x / len(data) for x in [sum(i) for i in zip(*list(data["req_ifrs_checked"]))]]

#Calculation of the percentages for ESRS relevant req_IDs
perc_comp_esrs = [x / len(data) for x in [sum(i) for i in zip(*list(data["req_esrs_checked"]))]]

#Calculation of the percentages for GRI relevant req_IDs
perc_comp_gri = [x / len(data) for x in [sum(i) for i in zip(*list(data["req_gri_checked"]))]]


#print(perc_comp_ifrs)
#print(perc_comp_esrs)
#print(perc_comp_gri)

We now add back that information to the master mapping table

In [ ]:
mapping["ifrs_req_perc"] = 0
mapping["esrs_req_perc"] = 0
mapping["gri_req_perc"] = 0

# Allocation of the percentage values to the individual req_IDs
c=0
for x in list(dict(mapping_ifrs).keys()):
  mapping = mapping.assign(ifrs_req_perc= np.select([mapping.req_ID == list(dict(mapping_ifrs).keys())[c]], [perc_comp_ifrs[c]], mapping.ifrs_req_perc))
  c +=1

c=0
for x in list(dict(mapping_esrs).keys()):
  mapping = mapping.assign(esrs_req_perc= np.select([mapping.req_ID == list(dict(mapping_esrs).keys())[c]], [perc_comp_esrs[c]], mapping.esrs_req_perc))
  c +=1

c=0
for x in list(dict(mapping_gri).keys()):
  mapping = mapping.assign(gri_req_perc= np.select([mapping.req_ID == list(dict(mapping_gri).keys())[c]], [perc_comp_gri[c]], mapping.gri_req_perc))
  c +=1


We calculate the absolute frequency of each `comp_ID` and transfer them to `mapping`.

In [ ]:
from collections import Counter

#Calculating the absolute frquency 
cou = Counter()
for xs in [list(data.loc[i, "compIDs"]) for i in data.index]:
  for x in set(xs):
    cou[x] += 1

#Convert to list of list
temp_l = [list(i) for i in cou.items()]

mapping["comp_abs_c"] = 0

#Assignment of the absolute frequency to the respective 'comp_ID' in 'mapping'
for i in temp_l:
  mapping.loc[mapping["comp_ID"] == i[0], ["comp_abs_c"]] = i[1]

mapping.to_excel("221011_mapping_updated.xlsx")

## Step 4. Collect the page refs and the links from the manual input file

In the web app we want to be able to locate the respective items. We collected page refs and links to the annual / sustainability reports. Now, we merge that to the company 'data' file.

In [ ]:
data_input_2 = pd.read_excel("221019_input_DAX40_AO.xlsx")
data_input_2.head(2)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Adidas,page-ref,Airbus,page-ref.1,BASF,page-ref.2,...,Siemens,page-ref.26,Siemens Healthineers,page-ref.27,Symrise,page-ref.28,Volkswagen,page-ref.29,Zalando,page-ref.30
0,NaN,NaN,NaN,NaN,DE000A1EWWW0,NaN,NL0000235190,NaN,DE000BASF111,NaN,...,DE0007236101,NaN,DE000SHL1006,NaN,DE000SYM9999,NaN,DE0007664039,NaN,DE000ZAL1111,NaN
1,NaN,NaN,primary link (default link),NaN,https://www.adidas-group.com/media/filer_publi...,NaN,https://www.airbus.com/sites/g/files/jlcbta136...,NaN,https://report.basf.com/2021/en/_assets/downlo...,NaN,...,https://new.siemens.com/content/dam/internet/s...,NaN,https://corporate.webassets.siemens-healthinee...,NaN,https://symrise.com/de/unternehmensbericht/202...,NaN,https://www.volkswagenag.com/presence/investor...,NaN,https://corporate.zalando.com/sites/default/fi...,NaN


In [ ]:
# Cleaning
data_input_2.drop(data_input_2.columns[list(range(4,(len(data_input_2.columns)),2))], axis=1, inplace=True)
data_input_2.drop(data_input_2.columns[[1,2,3]], axis=1, inplace=True)
data_input_2.drop([0,1,2], inplace=True)
data_input_2.reset_index(inplace=True, drop=True)
data_input_2.columns = list(data_input.columns)

In [ ]:
ref_page = []
for isin in list(data_input_2.columns[1:]):
    ref_page.append(data_input_2[isin].dropna().tolist())

data["compID_refs"] = ref_page

We now add the link information to that data table by importing a lookup table (which also has to be collected manually, see 2nd sheet in the Google Doc). In this step, we can add other information (e.g., size, Taxonomy).

In [ ]:
href_lookup_table = pd.read_excel("221014_lookup-table.xlsx")
print(href_lookup_table.shape)
href_lookup_table.head(2)

(243, 11)


,ID,Firm name,ISIN,Index,link_AR,link_SR,link_logo,mcap,Status,Responsible,by when
0,1.0,Adidas,DE000A1EWWW0,DAX,https://www.adidas-group.com/media/filer_publi...,n.a.,https://brandfetch.com/_next/image?url=https%3...,20421.2,Done,"Andreas, Inga, Victor",NaN
1,2.0,Airbus,NL0000235190,DAX,https://www.airbus.com/sites/g/files/jlcbta136...,n.a.,https://brandfetch.com/_next/image?url=https%3...,73433.2,Done,"Andreas, Inga, Victor",NaN


In [ ]:
# The lookup contains duplicates
href_lookup_table = href_lookup_table.loc[href_lookup_table["Status"] == "Done"]
print(href_lookup_table.shape)

(31, 11)


In [ ]:
data = pd.merge(data, href_lookup_table[["ISIN", "Index", "link_AR", "link_SR", "link_logo", "mcap"]], left_on=['isin'], right_on=['ISIN'])
data.drop("ISIN", axis=1, inplace=True)
data.head(2)

,name,isin,compIDs,req_ifrs_checked,req_ifrs,req_esrs_checked,req_esrs,req_gri_checked,req_gri,compID_refs,Index,link_AR,link_SR,link_logo,mcap
0,Adidas,DE000A1EWWW0,"[1.0, 67.0]","[False, False, False, False, False, False, Fal...",0.00,"[True, False, False, False, False, False, Fals...",0.019608,"[True, False, False, False, False, False, Fals...",0.071429,"[126.0, 126.0]",DAX,https://www.adidas-group.com/media/filer_publi...,n.a.,https://brandfetch.com/_next/image?url=https%3...,20421.2
1,Airbus,NL0000235190,"[1.0, 4.0, 5.0, 9.0, 34.0, 36.0, 40.0, 60.0, 6...","[True, False, False, True, False, False, False...",0.25,"[True, True, True, False, False, False, True, ...",0.166667,"[True, False, False, True, False, False, False...",0.285714,"[61.0, 105.0, 105.0, 61.0, 105.0, 105.0, 61.0,...",DAX,https://www.airbus.com/sites/g/files/jlcbta136...,n.a.,https://brandfetch.com/_next/image?url=https%3...,73433.2


In a last step, add the information if a page-ref belongs to the annual report (AR) or the sustainability report (SR). In the input sheet, this is indicated with an * after the page number. Python will read this not as a float, so I can create a list of bools if the information is in the AR (True) or not (False).

In [ ]:
data["compID_refs_locIsAR"] = [[True if y == "<class 'float'>" else False 
 for y in [str(type(x)) for x in data.iloc[i]["compID_refs"]]] for i in data.index]
data.head(2)

,name,isin,compIDs,req_ifrs_checked,req_ifrs,req_esrs_checked,req_esrs,req_gri_checked,req_gri,compID_refs,Index,link_AR,link_SR,link_logo,mcap,compID_refs_locIsAR
0,Adidas,DE000A1EWWW0,"[1.0, 67.0]","[False, False, False, False, False, False, Fal...",0.00,"[True, False, False, False, False, False, Fals...",0.019608,"[True, False, False, False, False, False, Fals...",0.071429,"[126.0, 126.0]",DAX,https://www.adidas-group.com/media/filer_publi...,n.a.,https://brandfetch.com/_next/image?url=https%3...,20421.2,"[True, True]"
1,Airbus,NL0000235190,"[1.0, 4.0, 5.0, 9.0, 34.0, 36.0, 40.0, 60.0, 6...","[True, False, False, True, False, False, False...",0.25,"[True, True, True, False, False, False, True, ...",0.166667,"[True, False, False, True, False, False, False...",0.285714,"[61.0, 105.0, 105.0, 61.0, 105.0, 105.0, 61.0,...",DAX,https://www.airbus.com/sites/g/files/jlcbta136...,n.a.,https://brandfetch.com/_next/image?url=https%3...,73433.2,"[True, True, True, True, True, True, True, Fal..."


In the next step, we clean the refs from the asterisk.

In [ ]:
data["compID_refs"] = [[float(i.replace("*", "")) if isinstance(i, str) else i for i in data.iloc[j]["compID_refs"]] for j in data.index]
data.loc[5, ["compID_refs", "compID_refs_locIsAR"]]

compID_refs            [26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26.0, 26....
compID_refs_locIsAR    [False, False, False, False, False, False, Fal...
Name: 5, dtype: object

The next step adds additional information about the firms and creates the data output

In [ ]:
corr = pd.read_excel("221020_input_DAX40_MM.xlsx")
corr = corr[["var_mcap" , "var_esgrating" , "var_scope1" , "var_scope2" , "var_co2int" , "var_reportingscore" , "var_rri"]]
data = data.join(corr)

In [ ]:
data.to_json("221019_data.json")
data.to_excel("221019_data.xlsx")

## Appendix

In [ ]:
#Ausgabe aller Informationen eines Unternehmens aus 'data'
#data_test.loc[lambda data: data["isin"] == "DE000A1EWWW0"]

#Werte in Dataframe setzen
#data_test = data
#data_test.loc[[0], ["Index"]] = "dax"
#data_test.iloc[[0:2], [1]]